## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
#from math import abs
data=pd.read_csv('data.csv')

data.head(30)

ModuleNotFoundError: No module named 'pymystem3'

### Вывод

Таблица содержит 12 столбцов и 21525 строк c типом данных в скобках:
    
    -children — количество детей в семье (int64)
    -days_employed — общий трудовой стаж в днях (float64)
    -dob_years — возраст клиента в годах (int64)
    -education — уровень образования клиента (object)
    -education_id — идентификатор уровня образования (int64)
    -family_status — семейное положение (object)
    -family_status_id — идентификатор семейного положения (int64)
    -gender — пол клиента (object)
    -income_type — тип занятости (object)
    -debt — имел ли задолженность по возврату кредитов (int64)
    -total_income — ежемесячный доход (float64)
    -purpose — цель получения кредита (object)

Количество значения в столбцах days_employed и total_income говорит о том, что есть пропуски, которые нужно устранить и понять причину их появления.

Каждая строка данных содержит в себе информацию том, какой пол, возраст (в годах), его семейное положение, количество детей, трудовой стаж в днях, ежемесячный доход, уровень образования, тип занятости и информация о цели кредита и задолжности по возврату кредитов.
Проблемы, которые надо решить- это пропуски Nan с данные в столбке days_employed и total_income (количество строк одинаковое). Возможно есть связь в источнике возникновения пропуска), элементы в столбце education привезти к одному регистру. Также встречаются значения с ссылкой.
Основные показатели для работы с гипотизей- это количество детей в семье, семейное положение, наличие задолженности по возврату кредита и доход.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
data[data['days_employed'].isnull()].head(10) # 10 значений столбца [days_employed] c пропуском

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [4]:
data[data['days_employed'].isnull()].count() # количество пропущенных (Nan) значений

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

В 2174 из 21525 строках пропущено значение о трудовом стаже в днях в столбце ['days_employed'].

## По замечанию исправление ниже

In [5]:
#data=data.dropna().reset_index(drop=True)
median_income_type=data.groupby('income_type')['total_income'].transform('mean')
median_income_type_m=data.groupby('income_type')['total_income'].mean()
#data['total_income'].fillna(data.loc[data['income_type']=='пенсионер']['total_income'].median()).head(30)
#data['totlal_new']=data['total_income'].fillna(data.groupby('income_type')['total_income'].mean())


In [6]:
median_income_type_m # Series data со средним доходом сгруппированных данных доход-род деятельности 

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

In [7]:
median_income_type # Series data по всей таблице со средними значениями дохода в завасимости от рода деятельности

0        161380.260488
1        161380.260488
2        161380.260488
3        161380.260488
4        137127.465690
             ...      
21520    202417.461462
21521    137127.465690
21522    161380.260488
21523    161380.260488
21524    161380.260488
Name: total_income, Length: 21525, dtype: float64

In [8]:
data['total_income'].fillna(median_income_type, inplace=True)# заменяем nan на средние значения 

In [9]:
data.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Произведены замены Nan в столбце доход по среднему доходу в группе доход-род деятельности.

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [11]:
data['days_employed'].isnull().sum() # количество пропусков в таблице

2174

In [12]:
del data['days_employed']# решил просто удалить, так как этот столбец не нужен для анализа

In [13]:
data.columns

Index(['children', 'dob_years', 'education', 'education_id', 'family_status',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

In [14]:
data.isna().sum()# Проверка на пропуски

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

## Конец исправления по замечанию

### Вывод

Так как в 2174 из 21525 строках пропущено значение о трудовом стаже в днях в столбце ['days_employed'] и ['total_income'], а это 4,7% от всех значений. Это количество не такое большое. Также мы не можем получить недостающие данные  о трудовом стаже и доходе. Эти значения можем удалить,а столбец ['days_employed'] убрать вовсе, чтобы не портили таблицу. Было замечено, что значения пропущены как в столбце ['days_employed'] так  и в столбце ['total_income'], причем с одинаковым индексом.
Возможные причины появления пропусков связяны с тем, что клиент не помнит свой трудовой стаж или он просто не работает, а про тип занятости мог соврать. Скорее всего эти пропуски не связяны с техническими проблемами, а человеческий фактор.

# В ходе обработки пропусков был удален столбец по стажу работы, т.к он не нужен для анализа. Пропуски по доходу были заменены на средние значения дохода по соответствующему роду деятельности.

### Замена типа данных

In [15]:
data['total_income']=data['total_income'].astype('int64') #переводим вещественный тип в целоцисленный

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


### Вывод

В столбце ['total_incom']  тип тоже был поменян с вещественного на целочисленный. Так просто приятнее на них смотреть.

### Обработка дубликатов

In [17]:
data['education'].value_counts() # количество дубликатов в столбце ['education']

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [18]:
data['education']=data['education'].str.lower() # приводим таблицу к нижнему регистру 

In [19]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [20]:
data.duplicated().sum()# Поиск дубликатов

71

In [21]:
data=data.drop_duplicates().reset_index(drop= True)# удаляем все дубликаты

In [22]:
data.duplicated().sum() #еще раз проверяем. Дубликатов нет

0

### Вывод

Дубликаты были в столбце ['education'] и отличались они только регистром. Поэтому я воспользовался функцией str.lower() и понизил весь текст по регистру. После этого ушли все дубликаты в столбце ['education']

### Лемматизация

In [23]:
m=Mystem() # присваиваем m библиотеку pymystem3 для лемматизации

In [24]:
data['purpose'].value_counts() # уникальные цели 

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Из этого списка уникальных целей выделяем 4 основные группы: недвижимость, автомобиль, образование, личные цели 

In [25]:
#В функцию lemmatization аргументом попадает цель purpose. Затем в lemma_tmp списов 
#лемматизарованых значений с помощию функции limmatize. Группа условий проверяет lemma_tmp и 
#определяет ее в одну из четырех: жилье, автомобиль, образование, личные цели.

def lemmatization (purpose):
    
    lemma_tmp=m.lemmatize(purpose)
              
    if ('жилье' in lemma_tmp) or ('недвижимость' in lemma_tmp):
        lemma_name_purpose='жилье'
        return lemma_name_purpose
    elif 'автомобиль' in lemma_tmp:
        lemma_name_purpose='автомобиль'
        return lemma_name_purpose
    elif 'образование' in lemma_tmp:
        lemma_name_purpose='образование'
        return lemma_name_purpose
    else:
        lemma_name_purpose='личные цели'
        return lemma_name_purpose

In [26]:
purp_lem=data['purpose'].apply(lemmatization) # Функция apply все значения закидывает в 
#функцию limmatization и присваивает purp_lem

In [27]:
data['purpose']=purp_lem #присваеваем переменную с леммами столбцу ['purpose'] 

In [28]:
data.head(5)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,жилье
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,жилье
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,личные цели


После лемматизации столбец выглядит понятнее, видно 4 категории целей, которыу можно удобно группировать и исследовать

# Убираем артифакты

In [29]:
data['gender'].value_counts()

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

In [30]:
data=data.drop(data[data['gender']=='XNA'].index) #удаляем в столбце {'gender'} значение XNA

In [31]:
data['gender'].value_counts()

F    14174
M     7279
Name: gender, dtype: int64

In [32]:
data['children'].value_counts()

 0     14090
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [33]:
data['children']=data['children'].replace(-1,1) # заменяем знечение в столбце ['children'] -1 на 1 и 20 на 2. 
data['children']=data['children'].replace(20,2) # скорее всего это были неправильно занесенные данные

In [34]:
data['children'].value_counts() # Теперь данные хорошие

0    14090
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [35]:
data['dob_years'].unique() 

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [36]:
data=data.drop(data[data['dob_years']==0].index) # Удаляем в столбце возраст значение 0. Оно явно ошибочное. 

In [37]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

### Категоризация данных

In [38]:
data.set_axis(['children(количество детей)','dob_years(возраст)','education(образование)','education_id','family_status(семейный статус)','family_status_id','gender(пол)','income_type(род деятельности)','debt(просрочки)','total_income(уровень дохода)','purpose(цель кредита)'],axis='columns',inplace=True) # меняем название столбцов

In [39]:
data.head(5)

,children(количество детей),dob_years(возраст),education(образование),education_id,family_status(семейный статус),family_status_id,gender(пол),income_type(род деятельности),debt(просрочки),total_income(уровень дохода),purpose(цель кредита)
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,жилье
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,жилье
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,личные цели


In [40]:
data.sort_values(by='dob_years(возраст)') # сортируем по столбцу возраст по возрастанию

,children(количество детей),dob_years(возраст),education(образование),education_id,family_status(семейный статус),family_status_id,gender(пол),income_type(род деятельности),debt(просрочки),total_income(уровень дохода),purpose(цель кредита)
8307,0,19,среднее,1,Не женат / не замужем,4,F,сотрудник,0,68524,образование
4050,0,19,среднее,1,гражданский брак,1,M,сотрудник,0,161380,личные цели
15624,0,19,неоконченное высшее,2,гражданский брак,1,F,компаньон,0,167211,автомобиль
2725,0,19,среднее,1,Не женат / не замужем,4,F,компаньон,0,131967,жилье
12027,0,19,среднее,1,Не женат / не замужем,4,M,сотрудник,1,93343,жилье
...,...,...,...,...,...,...,...,...,...,...,...
3458,0,74,среднее,1,женат / замужем,0,M,пенсионер,0,54754,жилье
12297,0,74,среднее,1,женат / замужем,0,M,компаньон,0,124830,жилье
4891,0,74,высшее,0,женат / замужем,0,F,пенсионер,0,134935,жилье
2557,0,74,среднее,1,женат / замужем,0,F,пенсионер,0,42927,автомобиль


Возраст варьируется от 19 до 75 лет.

Категоризируем на возрастные группы:

In [41]:
# Функция для разбивки на категории в зависимости от возраста.
def dob_years_group(age):
    if 19<=age<=35:
        return 'молодые'
    if 36<age<59:
        return 'взрослые'
    return 'пенсионеры'

In [46]:
dob_years_group(60) # проверочка!!!

'пенсионеры'

In [41]:
dob_years_group=data['dob_years(возраст)'].apply(dob_years_group)

In [42]:
data['dob_years(возраст)']=dob_years_group
data

,children(количество детей),dob_years(возраст),education(образование),education_id,family_status(семейный статус),family_status_id,gender(пол),income_type(род деятельности),debt(просрочки),total_income(уровень дохода),purpose(цель кредита)
0,1,взрослые,высшее,0,женат / замужем,0,F,сотрудник,0,253875,жилье
1,1,пенсионеры,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,молодые,среднее,1,женат / замужем,0,M,сотрудник,0,145885,жилье
3,3,молодые,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,взрослые,среднее,1,гражданский брак,1,F,пенсионер,0,158616,личные цели
...,...,...,...,...,...,...,...,...,...,...,...
21449,1,взрослые,среднее,1,гражданский брак,1,F,компаньон,0,224791,жилье
21450,0,пенсионеры,среднее,1,женат / замужем,0,F,пенсионер,0,155999,автомобиль
21451,1,взрослые,среднее,1,гражданский брак,1,M,сотрудник,1,89672,жилье
21452,3,взрослые,среднее,1,женат / замужем,0,M,сотрудник,1,244093,автомобиль


In [43]:
data.sort_values(by='total_income(уровень дохода)')

,children(количество детей),dob_years(возраст),education(образование),education_id,family_status(семейный статус),family_status_id,gender(пол),income_type(род деятельности),debt(просрочки),total_income(уровень дохода),purpose(цель кредита)
14555,0,взрослые,среднее,1,женат / замужем,0,F,пенсионер,1,20667,жилье
12983,0,взрослые,среднее,1,гражданский брак,1,M,пенсионер,0,21205,образование
16138,1,взрослые,среднее,1,женат / замужем,0,M,сотрудник,0,21367,автомобиль
1598,0,пенсионеры,среднее,1,гражданский брак,1,M,пенсионер,0,21695,личные цели
14247,0,пенсионеры,среднее,1,женат / замужем,0,F,пенсионер,0,21895,жилье
...,...,...,...,...,...,...,...,...,...,...,...
17137,0,взрослые,высшее,0,гражданский брак,1,M,компаньон,0,1711309,личные цели
20742,0,пенсионеры,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1715018,жилье
9159,1,молодые,среднее,1,гражданский брак,1,M,сотрудник,0,1726276,образование
19548,1,взрослые,высшее,0,женат / замужем,0,M,компаньон,1,2200852,жилье


Уровень дохода лужит в промежутке от 20 тр руб и до 2.26 млн руб.

In [44]:
# Функция для категоризации по уровню дохода
def total_income_group(income):
    if 20000<=income<70000:
        return "низкий доход"
    if 70001<=income<120000:
        return 'средний доход'
    if 120001<=income<200000:
        return 'высокий доход'
    return 'очень высокий доход'

In [1]:
total_income_group(70000)

NameError: name 'total_income_group' is not defined

In [45]:
total_income_gr=data['total_income(уровень дохода)'].apply(total_income_group)

In [46]:
data['total_income(уровень дохода)']=total_income_gr
data.head(10)

,children(количество детей),dob_years(возраст),education(образование),education_id,family_status(семейный статус),family_status_id,gender(пол),income_type(род деятельности),debt(просрочки),total_income(уровень дохода),purpose(цель кредита)
0,1,взрослые,высшее,0,женат / замужем,0,F,сотрудник,0,очень высокий доход,жилье
1,1,пенсионеры,среднее,1,женат / замужем,0,F,сотрудник,0,средний доход,автомобиль
2,0,молодые,среднее,1,женат / замужем,0,M,сотрудник,0,высокий доход,жилье
3,3,молодые,среднее,1,женат / замужем,0,M,сотрудник,0,очень высокий доход,образование
4,0,взрослые,среднее,1,гражданский брак,1,F,пенсионер,0,высокий доход,личные цели
5,0,молодые,высшее,0,гражданский брак,1,M,компаньон,0,очень высокий доход,жилье
6,0,взрослые,высшее,0,женат / замужем,0,F,компаньон,0,очень высокий доход,жилье
7,0,взрослые,среднее,1,женат / замужем,0,M,сотрудник,0,высокий доход,образование
8,2,молодые,высшее,0,гражданский брак,1,F,сотрудник,0,средний доход,личные цели
9,0,взрослые,среднее,1,женат / замужем,0,M,сотрудник,0,высокий доход,жилье


### Вывод

В процессе категоризации были выбраны две группы, которые я разбивал на категории. Это столбец Возраст и столбец Доход. Категоризация проходила с помощью функиции с группой условий и метода apply. Эти столбцы были выбраны для категоризации, т.к. они используются в дальнейшем исследовании и это очень удобный вид для представления этих данных

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [47]:
chil_debt=data.groupby('children(количество детей)').agg({'debt(просрочки)':['count','sum']}) # агрегированная функция

In [48]:
chil_debt

debt(просрочки)      
                                     count   sum
children(количество детей)                      
0                                    14021  1058
1                                     4839   442
2                                     2114   202
3                                      328    27
4                                       41     4
5                                        9     0

In [49]:
chil_quality=chil_debt['debt(просрочки)']['sum']/chil_debt['debt(просрочки)']['count']

In [50]:
chil_quality # Подсет доли

children(количество детей)
0    0.075458
1    0.091341
2    0.095553
3    0.082317
4    0.097561
5    0.000000
dtype: float64

### Вывод

Из этих данных видно, что возврат кредит связан с количеством детей. Больше всего просрочек у людей с 1, 2,3,4 детьми. Нет росрочек в тех у кого 5 детей. У кого их вообще нет- просрочек меньше. Скорее всего это связано с финансовой нагрузкой

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [51]:
family_grouped=data.groupby('family_status(семейный статус)').agg({'debt(просрочки)':['count','sum']}) 
family_debt_quality=family_grouped['debt(просрочки)']['sum']/family_grouped['debt(просрочки)']['count']

In [52]:
family_debt_quality

family_status(семейный статус)
Не женат / не замужем    0.097709
в разводе                0.071730
вдовец / вдова           0.064990
гражданский брак         0.093485
женат / замужем          0.075427
dtype: float64

Посчитали долю просрочек по кредиту в зависимости от статуса человека. Сумму и количество посчитали с помощью гуппивки и метода count и sum в agg()

### Вывод

На основании среза данных "семейный статус" и "просрочек по кредиту", можно увидеть интересные данные:
- Люди, которые были в официальных отношениях, а теперь по-одиночке имеют меньше просрочек по кредиту. Это видно на примере данных "вдовец/вдова"- доля просрочек на всех "вдовцов/вдов" составляет- 0.06. Похожие результаты можно увидеть и у людей, которые в "разводе". Доля просрочек на всех разведенных составляет - 0.07. Я это могу объяснить тем, что человек, который был в официальном браке, а в дальнейшем развелся или потерял мужа/жену, инеет высокий уровень ответственности. В их жизни приходилось принимать серьезные решения.  
-  Доля просрочек на всех женатых/ замужем составляет- 0.075, но и этот показатель не самых плохой. Можно сказать, что доля просрочек у "женат / замужем" и в "разводе" почти одинакова;
- Что касается людей в "гражданском браке" и "не женатых / не замужем", то тут самый плохой показатель. Хуже всего этот показатель у "не женат / не замужем"- доля просрочек - 0.1. Думаю, что это связано с тем, что человек не готов к серьезному развитию (отсутствие семьи), а значит нет желания брать ответственность. Кредитную тоже!


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [53]:
total_income_grouped=data.groupby('total_income(уровень дохода)').agg({'debt(просрочки)':['sum','count']})
total_income_quality=total_income_grouped['debt(просрочки)']['sum']/total_income_grouped['debt(просрочки)']['count']

In [54]:
total_income_grouped['debt(просрочки)']

,sum,count
total_income(уровень дохода),,
высокий доход,798,9005
низкий доход,101,1469
очень высокий доход,387,5539
средний доход,447,5339


In [55]:
total_income_quality

total_income(уровень дохода)
высокий доход          0.088617
низкий доход           0.068754
очень высокий доход    0.069868
средний доход          0.083724
dtype: float64

Посчитали долю просрочек по кредиту в зависимости от дохода. Сумму и количество посчитали с помощью гуппивки и метода count и sum в agg()

In [56]:
data_pivot=data.pivot_table(index=['dob_years(возраст)'],columns='total_income(уровень дохода)',values='debt(просрочки)',aggfunc='sum')
data_pivot

total_income(уровень дохода),высокий доход,низкий доход,очень высокий доход,средний доход
dob_years(возраст),,,,
взрослые,368,41,205,225
молодые,334,35,147,182
пенсионеры,96,25,35,40


Сводная таблица просрочкам кредита в зависимости от возраста и уровня дохода. Тут можно увидеть какой возраст имеет больше всего просрочек.

In [57]:
data_pivot=data.pivot_table(index=['income_type(род деятельности)','dob_years(возраст)'],columns='total_income(уровень дохода)',values='debt(просрочки)',aggfunc='sum')

In [58]:
data_pivot

total_income(уровень дохода)                      высокий доход  низкий доход  \
income_type(род деятельности) dob_years(возраст)                                
безработный                   взрослые                      NaN           NaN   
                              молодые                       NaN           1.0   
в декрете                     взрослые                      NaN           1.0   
госслужащий                   взрослые                     21.0           0.0   
                              молодые                      12.0           4.0   
                              пенсионеры                    3.0           0.0   
компаньон                     взрослые                     60.0           5.0   
                              молодые                      76.0           6.0   
                              пенсионеры                   11.0           0.0   
пенсионер                     взрослые                     44.0           7.0   
                              молодые                       1.0           0.0   
                              пенсионеры                   55.0          22.0   
предприниматель               взрослые                      NaN           NaN   
                              молодые                       NaN           NaN   
сотрудник                     взрослые                    243.0          28.0   
                              молодые                     245.0          24.0   
                              пенсионеры                   27.0           3.0   
студент                       молодые                       NaN           NaN   

total_income(уровень дохода)                      очень высокий доход  \
income_type(род деятельности) dob_years(возраст)                        
безработный                   взрослые                            0.0   
                              молодые                             NaN   
в декрете                     взрослые                            NaN   
госслужащий                   взрослые                           10.0   
                              молодые                             8.0   
                              пенсионеры                          0.0   
компаньон                     взрослые                           76.0   
                              молодые                            55.0   
                              пенсионеры                         11.0   
пенсионер                     взрослые                           18.0   
                              молодые                             0.0   
                              пенсионеры                         13.0   
предприниматель               взрослые                            0.0   
                              молодые                             0.0   
сотрудник                     взрослые                          101.0   
                              молодые                            84.0   
                              пенсионеры                         11.0   
студент                       молодые                             NaN   

total_income(уровень дохода)                      средний доход  
income_type(род деятельности) dob_years(возраст)                 
безработный                   взрослые                      NaN  
                              молодые                       NaN  
в декрете                     взрослые                      NaN  
госслужащий                   взрослые                     13.0  
                              молодые                      13.0  
                              пенсионеры                    2.0  
компаньон                     взрослые                     35.0  
                              молодые                      36.0  
                              пенсионеры                    4.0  
пенсионер                     взрослые                     31.0  
                              молодые                       0.0  
                              пенсионеры                   2

Сводная таблица просрочек по кредиту в зависимости от трудового стажа, возраста и дохода. Можно видеть,что много просрочек по кредиту у служащих молодых и взрослых.

### Вывод

- Как разные цели кредита влияют на его возврат в срок?

In [59]:
purpose_grouped=data.groupby('purpose(цель кредита)').agg({'debt(просрочки)':['sum','count']})
purpose_convers=purpose_grouped['debt(просрочки)']['sum']/purpose_grouped['debt(просрочки)']['count']
purpose_convers

purpose(цель кредита)
автомобиль     0.093371
жилье          0.072378
личные цели    0.079654
образование    0.092616
dtype: float64

Посчитали долю просрочек по кредиту в зависимости от цели кредита. Сумму и количество посчитали с помощью гуппивки и метода count и sum в agg()

### Вывод

Из полученных данных среза "цель кредита и просрочи можно сделать вывод, что:
- больше всего просрочек у кредитов на автомобиль и образование. Доля их получается 0.092. Можно это объяснить тем, что на рынке автокредитования создаются очень привлекательные предложения и человек порой берет кредит не оценевая аддекватно свои возможности.  
- Кредит на личные цели и кредит на недвижимось имеют меньше просрочек. Человек, выбирая ипотеку, хорошо просчитывает свои возможности, так как срок платежа очень большой. 

### Шаг 4. Общий вывод

Из моих данных я могу сделать вывод, что самыми надежными заемщиками являются люди, которые берут ипотеку, с низким или очень высоким уровнем заработка, имеющие детей (чем больше, тем лучше), состоящие в официальном браке(лучше вдовец или вдова)